In [1]:
%reload_ext sql
%config SqlMagic.displaylimit = None
%sql sqlite:///library.db

displaylimit: Value None will be treated as 0 (no limit)

Connecting to 'sqlite:///library.db'

In [105]:
%%sql
-- Ensure foreign keys are enabled (Run separately before this script in SQLite)
PRAGMA foreign_keys = ON;

CREATE TABLE
  IF NOT EXISTS events (
    eventID INTEGER PRIMARY KEY,
    eventName TEXT NOT NULL,
    eventType TEXT NOT NULL,
    description TEXT,
    audienceType TEXT,
    eventDate TEXT CHECK (eventDate LIKE '____-__-__'),
    startTime TIME NOT NULL, -- Changed from TEXT to TIME for proper comparison
    endTime TIME NOT NULL,
    roomNum INTEGER NOT NULL,
    FOREIGN KEY (roomNum) REFERENCES socialRooms (roomNum) ON DELETE CASCADE,
    CHECK (startTime < endTime)
  );

CREATE TABLE
  IF NOT EXISTS item (
    itemID INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT,
    publicationYear INTEGER,
    genre TEXT,
    itemType TEXT,
    isAvailable INTEGER CHECK (isAvailable IN (0, 1)),
    location TEXT
  );

CREATE TABLE
  IF NOT EXISTS borrowedBy (
    borrowingID INTEGER PRIMARY KEY AUTOINCREMENT,
    itemID INTEGER NOT NULL,
    userID INTEGER NOT NULL,
    borrowDate TEXT CHECK (borrowDate LIKE '____-__-__'),
    returnDate TEXT CHECK (returnDate LIKE '____-__-__'),
    fineAmnt REAL DEFAULT 0.0,
    FOREIGN KEY (itemID) REFERENCES item (itemID) ON DELETE CASCADE,
    FOREIGN KEY (userID) REFERENCES patron (userID) ON DELETE CASCADE,
    CHECK (borrowDate < returnDate)
  );

CREATE TABLE
  IF NOT EXISTS socialRooms (
    roomNum INTEGER PRIMARY KEY,
    isAvailable INTEGER CHECK (isAvailable IN (0, 1)),
    capacity INTEGER CHECK (capacity > 0)
  );

CREATE TABLE
  IF NOT EXISTS personnel (
    staffID INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    POSITION TEXT NOT NULL,
    phoneNum NUMERIC,
    isActive INTEGER CHECK (isActive IN (0, 1))
  );

CREATE TABLE
  IF NOT EXISTS patron (
    userID INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    phoneNum NUMERIC UNIQUE
  );

CREATE TABLE
  IF NOT EXISTS futureItems (
    itemID INTEGER PRIMARY KEY,
    expectedArrivalDate TEXT CHECK (expectedArrivalDate LIKE '____-__-__'),
    FOREIGN KEY (itemID) REFERENCES item (itemID) ON DELETE CASCADE
  );

CREATE TABLE
  IF NOT EXISTS registerEvent (
    eventID INTEGER,
    userID INTEGER,
    PRIMARY KEY (eventID, userID),
    FOREIGN KEY (eventID) REFERENCES events (eventID) ON DELETE CASCADE,
    FOREIGN KEY (userID) REFERENCES patron (userID) ON DELETE CASCADE
  );

CREATE TABLE
  IF NOT EXISTS askLibrarian (
    requestID INTEGER PRIMARY KEY AUTOINCREMENT,
    userID INTEGER NOT NULL,
    staffID INTEGER NOT NULL,
    requestTime TEXT CHECK (requestTime LIKE '____-__-__ __:__:__'),
    description TEXT NOT NULL,
    status INTEGER CHECK (status IN (0, 1, 2)),
    FOREIGN KEY (staffID) REFERENCES personnel (staffID) ON DELETE SET NULL,
    FOREIGN KEY (userID) REFERENCES patron (userID) ON DELETE CASCADE
  );

CREATE TABLE
  IF NOT EXISTS donations (
    donationID INTEGER PRIMARY KEY AUTOINCREMENT,
    userID INTEGER NOT NULL,
    itemID INTEGER NOT NULL,
    donationDate TEXT CHECK (donationDate LIKE '____-__-__'),
    FOREIGN KEY (itemID) REFERENCES item (itemID) ON DELETE CASCADE,
    FOREIGN KEY (userID) REFERENCES patron (userID) ON DELETE CASCADE
  );

CREATE TABLE
  IF NOT EXISTS volunteers (
    userID INTEGER NOT NULL,
    staffID INTEGER NOT NULL,
    PRIMARY KEY (userID, staffID),
    FOREIGN KEY (staffID) REFERENCES personnel (staffID) ON DELETE CASCADE,
    FOREIGN KEY (userID) REFERENCES patron (userID) ON DELETE CASCADE
  );

Running query in 'sqlite:///libray.db'

++
||
++
++

In [59]:
%%sql
PRAGMA table_info(item);


Running query in 'sqlite:///libray.db'

cid,name,type,notnull,dflt_value,pk
0,itemID,INTEGER,0,None,1
1,title,TEXT,1,None,0
2,author,TEXT,0,None,0
3,publicationYear,INTEGER,0,None,0
4,genre,TEXT,0,None,0
5,itemType,TEXT,0,None,0
6,isAvailable,INTEGER,0,None,0
7,location,TEXT,0,None,0


In [49]:
%%sql 
-- SELECT name FROM sqlite_master WHERE type='table';

--works
SELECT * from item;
-- SELECT COUNT(*) from borrowedBy;
-- SELECT COUNT(*) from item;
-- SELECT COUNT(*) from patron;
-- SELECT COUNT(*) from socialRooms;
-- SELECT COUNT(*) from events;
-- SELECT COUNT(*) from donations;
-- SELECT COUNT(*) from askLibrian;
-- SELECT COUNT(*) from volunteers;





Running query in 'sqlite:///libray.db'

++
||
++
++

In [103]:
%%sql
PRAGMA foreign_keys = OFF;

DROP TABLE IF EXISTS registerEvent;
DROP TABLE IF EXISTS volunteers;
DROP TABLE IF EXISTS donations;
DROP TABLE IF EXISTS askLibrarian;
DROP TABLE IF EXISTS futureItems;
DROP TABLE IF EXISTS borrowedBy;
DROP TABLE IF EXISTS events;
DROP TABLE IF EXISTS socialRooms;
DROP TABLE IF EXISTS personnel;
DROP TABLE IF EXISTS patron;
DROP TABLE IF EXISTS item;

Running query in 'sqlite:///libray.db'

++
||
++
++

In [107]:
%%sql
-- Insert sample data for socialRooms
INSERT INTO socialRooms (roomNum, isAvailable, capacity) VALUES
(500, 1, 50),
(501, 1, 100),
(502, 1, 30),
(503, 1, 40),
(504, 1, 60),
(505, 1, 70),
(506, 1, 80),
(507, 1, 90),
(508, 1, 100),
(509, 1, 30);

Running query in 'sqlite:///libray.db'

10 rows affected.

++
||
++
++

In [109]:
%%sql
-- Insert sample data for events
INSERT INTO events (eventID, eventName, eventType, description, audienceType, eventDate, startTime, endTime, roomNum) VALUES
(400, 'Book Club Meeting', 'Discussion', 'Monthly book club meetup', 'Adults', '2024-04-01', '18:00:00', '20:00:00', 500),
(401, 'Tech Talk', 'Seminar', 'AI and future trends', 'General', '2024-04-05', '14:00:00', '16:00:00', 501),
(402, 'Story Time', 'Reading', 'Children storytelling session', 'Children', '2024-04-10', '10:00:00', '11:30:00', 502),
(403, 'Poetry Night', 'Performance', 'Open mic poetry', 'Teens', '2024-04-12', '19:00:00', '21:00:00', 503),
(404, 'Chess Tournament', 'Competition', 'Annual chess contest', 'General', '2024-04-15', '10:00:00', '15:00:00', 504),
(405, 'Coding Workshop', 'Workshop', 'Intro to Python', 'Teens', '2024-04-18', '13:00:00', '16:00:00', 505),
(406, 'Author Meet & Greet', 'Discussion', 'Q&A with famous author', 'General', '2024-04-22', '17:00:00', '19:00:00', 506),
(407, 'Film Screening', 'Entertainment', 'Classic movie night', 'General', '2024-04-25', '20:00:00', '22:30:00', 507),
(408, 'Library Game Night', 'Social', 'Board games and card games for all ages', 'General', '2024-04-28', '17:00:00', '20:00:00', 508),
(409, 'Science Fair', 'Educational', 'Annual community science fair', 'Teens', '2024-04-30', '10:00:00', '15:00:00', 509);
-- Insert sample data for item

INSERT INTO item (
    title,
    author,
    publicationYear,
    genre,
    itemType,
    isAvailable,
    location
) VALUES
    ('To Kill a Mockingbird', 'Harper Lee', 1960, 'Fiction', 'Book', 1, 'Shelf A1'),
    ('1984', 'George Orwell', 1949, 'Dystopian', 'Book', 1, 'Shelf A2'),
    ('The Catcher in the Rye', 'J.D. Salinger', 1951, 'Classic', 'Book', 1, 'Shelf B1'),
    ('Sapiens', 'Yuval Noah Harari', 2011, 'Non-fiction', 'Book', 1, 'Shelf C1'),
    ('Inception', 'Christopher Nolan', 2010, 'Sci-Fi', 'DVD', 1, 'Shelf D1'),
    ('The Matrix', 'The Wachowskis', 1999, 'Sci-Fi', 'DVD', 1, 'Shelf D2'),
    ('Interstellar', 'Christopher Nolan', 2014, 'Sci-Fi', 'DVD', 1, 'Shelf D3'),
    ('The Godfather', 'Mario Puzo', 1972, 'Crime', 'DVD', 1, 'Shelf D4'),
    ('Harry Potter and the Sorcerer''s Stone', 'J.K. Rowling', 1997, 'Fantasy', 'Book', 1, 'Shelf B2'),
    ('The Great Gatsby', 'F. Scott Fitzgerald', 1925, 'Classic', 'Book', 1, 'Shelf B3');                     


-- -- Insert sample data for personnel
INSERT INTO personnel (staffID, firstName, lastName, position, phoneNum, isActive) VALUES
(200, 'Alice', 'Johnson', 'Librarian', '1234567890', 1),
(201, 'Bob', 'Smith', 'Assistant', '1234567891', 1),
(202, 'Charlie', 'Brown', 'Manager', '1234567892', 1),
(203, 'David', 'White', 'Technician', '1234567893', 1),
(204, 'Eve', 'Black', 'Custodian', '1234567894', 1),
(205, 'Frank', 'Green', 'Security', '1234567895', 1),
(206, 'Grace', 'Blue', 'Archivist', '1234567896', 1),
(207, 'Hank', 'Red', 'IT Support', '1234567897', 1),
(208, 'Ivy', 'Yellow', 'Researcher', '1234567898', 1),
(209, 'Jack', 'Gray', 'Volunteer Coordinator', '1234567899', 1);
-- -- Insert sample data for user
INSERT INTO patron (firstName, lastName, phoneNum) VALUES
('John', 'Doe', '9876543210'),
('Jane', 'Doe', '9876543211'),
('Emily', 'Clark', '9876543212'),
('Michael', 'Brown', '9876543213'),
('Sarah', 'Johnson', '9876543214'),
('Chris', 'Lee', '9876543215'),
('Anna', 'Taylor','9876543216'),
('David', 'Wilson', '9876543217'),
('Emma', 'Martinez','9876543218'),
('James', 'Anderson', '9876543219');

Running query in 'sqlite:///libray.db'

10 rows affected.

10 rows affected.

10 rows affected.

10 rows affected.

++
||
++
++

In [111]:
%%sql
SELECT * FROM patron;

Running query in 'sqlite:///libray.db'

userID,firstName,lastName,phoneNum
1,John,Doe,9876543210
2,Jane,Doe,9876543211
3,Emily,Clark,9876543212
4,Michael,Brown,9876543213
5,Sarah,Johnson,9876543214
6,Chris,Lee,9876543215
7,Anna,Taylor,9876543216
8,David,Wilson,9876543217
9,Emma,Martinez,9876543218
10,James,Anderson,9876543219


In [115]:
%%sql
SELECT * FROM item;

Running query in 'sqlite:///libray.db'

itemID,title,author,publicationYear,genre,itemType,isAvailable,location
1,To Kill a Mockingbird,Harper Lee,1960,Fiction,Book,1,Shelf A1
2,1984,George Orwell,1949,Dystopian,Book,1,Shelf A2
3,The Catcher in the Rye,J.D. Salinger,1951,Classic,Book,1,Shelf B1
4,Sapiens,Yuval Noah Harari,2011,Non-fiction,Book,1,Shelf C1
5,Inception,Christopher Nolan,2010,Sci-Fi,DVD,1,Shelf D1
6,The Matrix,The Wachowskis,1999,Sci-Fi,DVD,1,Shelf D2
7,Interstellar,Christopher Nolan,2014,Sci-Fi,DVD,1,Shelf D3
8,The Godfather,Mario Puzo,1972,Crime,DVD,1,Shelf D4
9,Harry Potter and the Sorcerer's Stone,J.K. Rowling,1997,Fantasy,Book,1,Shelf B2
10,The Great Gatsby,F. Scott Fitzgerald,1925,Classic,Book,1,Shelf B3


In [127]:
%%sql
SELECT * FROM personnel;

Running query in 'sqlite:///libray.db'

staffID,firstName,lastName,POSITION,phoneNum,isActive
200,Alice,Johnson,Librarian,1234567890,1
201,Bob,Smith,Assistant,1234567891,1
202,Charlie,Brown,Manager,1234567892,1
203,David,White,Technician,1234567893,1
204,Eve,Black,Custodian,1234567894,1
205,Frank,Green,Security,1234567895,1
206,Grace,Blue,Archivist,1234567896,1
207,Hank,Red,IT Support,1234567897,1
208,Ivy,Yellow,Researcher,1234567898,1
209,Jack,Gray,Volunteer Coordinator,1234567899,1


In [119]:
%%sql
-- Insert sample data for borrowedBy
INSERT INTO borrowedBy (borrowingID, itemID, userID, borrowDate, returnDate, fineAmnt) VALUES
(1, 1, 1, '2024-03-01', '2024-03-15', 0.0),
(2, 2, 2, '2024-03-02', '2024-03-16', 0.0),
(3, 3, 3, '2024-03-05', '2024-03-19', 0.0),
(4, 4, 4, '2024-03-10', '2024-03-24', 0.0),
(5, 5, 5, '2024-03-15', '2024-03-29', 0.0),
(6, 6, 6, '2024-03-20', '2024-04-03', 1.5),
(7, 7, 7, '2024-03-22', '2024-04-05', 0.0),
(8, 8, 8, '2024-03-25', '2024-04-08', 2.0),
(9, 9, 9, '2024-03-27', '2024-04-10', 0.0),
(10, 10, 10, '2024-03-30', '2024-04-13', 0.0);

Running query in 'sqlite:///libray.db'

10 rows affected.

++
||
++
++

In [129]:
%%sql
INSERT INTO volunteers (userID, staffID) VALUES
(1, 200),
(2, 201),
(3, 202),
(4, 203),
(5, 204),
(6, 205),
(7, 206),
(8, 207),
(9, 208),
(10, 209);

INSERT INTO donations (donationID, userID, itemID, donationDate) VALUES
(500, 1, 1, '2024-03-01'),
(501, 2, 2, '2024-03-05'),
(502, 3, 3, '2024-03-08'),
(503, 4, 4, '2024-03-12'),
(504, 5, 5, '2024-03-15'),
(505, 6, 6, '2024-03-18'),
(506, 7, 7, '2024-03-22'),
(507, 8, 8, '2024-03-25'),
(508, 9, 9, '2024-03-28'),
(509, 10, 10, '2024-03-30');

INSERT INTO askLibrarian (requestID, userID, staffID, requestTime, description, status) VALUES
(600, 1, 200, '2024-03-01 10:15:00', 'Inquiry about book availability', 0),
(601, 2, 201, '2024-03-02 14:30:00', 'Help with research materials', 1),
(602, 3, 202, '2024-03-03 09:45:00', 'Issue with borrowed book return', 2),
(603, 4, 203, '2024-03-04 13:10:00', 'Request for book recommendation', 0),
(604, 5, 204, '2024-03-05 16:20:00', 'Guidance on digital resources', 1),
(605, 6, 205, '2024-03-06 11:05:00', 'Request for room booking', 2),
(606, 7, 206, '2024-03-07 12:40:00', 'Assistance with library membership', 0),
(607, 8, 207, '2024-03-08 15:00:00', 'Lost item inquiry', 1),
(608, 9, 208, '2024-03-09 17:25:00', 'Help with citation formatting', 2),
(609, 10, 209, '2024-03-10 08:50:00', 'Request for event details', 0);

INSERT INTO registerEvent (eventID, userID) VALUES
(400, 1),
(401, 2),
(402, 3),
(403, 4),
(404, 5),
(405, 6),
(406, 7),
(407, 8),
(408, 9),
(409, 10);
INSERT INTO futureItems (itemID, expectedArrivalDate) VALUES
(1, '2024-05-01'),
(2, '2024-05-05'),
(3, '2024-05-10'),
(4, '2024-05-15'),
(5, '2024-05-20'),
(6, '2024-05-25'),
(7, '2024-06-01'),
(8, '2024-06-05'),
(9, '2024-06-10'),
(10, '2024-06-15');

Running query in 'sqlite:///libray.db'

10 rows affected.

10 rows affected.

10 rows affected.

10 rows affected.

10 rows affected.

++
||
++
++

In [21]:
%%sql
CREATE TABLE donations_new (
    donationID INTEGER PRIMARY KEY AUTOINCREMENT,
    userID INTEGER NOT NULL,
    itemID INTEGER NOT NULL,
    donationDate TEXT CHECK (donationDate LIKE '____-__-__'),
    FOREIGN KEY (itemID) REFERENCES item (itemID) ON DELETE CASCADE,
    FOREIGN KEY (userID) REFERENCES patron (userID) ON DELETE CASCADE
);

-- Step 2: Copy data without borrowingID (fresh new IDs will be generated)
INSERT INTO donations_new (userID, itemID, donationDate)
SELECT userID, itemID, donationDate FROM donations;

-- Step 3: Drop the old table
DROP TABLE donations;

-- Step 4: Rename the new table
ALTER TABLE donations_new RENAME TO donations;

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [7]:
%%sql
UPDATE volunteers
SET staffID = userID
    WHERE userID BETWEEN 1 AND 10;

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [23]:
%%sql
SELECT * FROM donations;

Running query in 'sqlite:///library.db'

donationID,userID,itemID,donationDate
1,1,1,2024-03-01
2,2,2,2024-03-05
3,3,3,2024-03-08
4,4,4,2024-03-12
5,5,5,2024-03-15
6,6,6,2024-03-18
7,7,7,2024-03-22
8,8,8,2024-03-25
9,9,9,2024-03-28
10,10,10,2024-03-30


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++